<a href="https://www.kaggle.com/code/dimassp1/f1-0-962-pytorch?scriptVersionId=258636494" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

from torch import nn
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
# import pytorch_optimizer as optim1


from pytorch_lightning import LightningModule
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import pytorch_lightning as pl
from torchmetrics.classification import F1Score, MulticlassCohenKappa, Accuracy
pl.seed_everything(42)

42

In [2]:
df = pd.read_csv('/kaggle/input/multiclass-diabetes-dataset/Multiclass Diabetes Dataset/Multiclass Diabetes Dataset.csv')
df

,Gender,AGE,Urea,Cr,HbA1c,Chol,TG,HDL,LDL,VLDL,BMI,Class
0,0,50,4.7,46,4.9,4.2,0.9,2.4,1.4,0.5,24.0,0
1,1,26,4.5,62,4.9,3.7,1.4,1.1,2.1,0.6,23.0,0
2,1,33,7.1,46,4.9,4.9,1.0,0.8,2.0,0.4,21.0,0
3,0,45,2.3,24,4.0,2.9,1.0,1.0,1.5,0.4,21.0,0
4,0,50,2.0,50,4.0,3.6,1.3,0.9,2.1,0.6,24.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
259,0,61,24.0,401,7.0,4.3,2.9,0.4,2.7,1.3,30.0,2
260,0,61,24.0,401,7.0,4.3,2.9,0.4,2.7,1.3,36.6,2
261,1,60,20.8,800,9.0,2.3,1.1,0.9,0.9,0.5,33.0,2
262,1,56,20.8,800,9.0,4.6,2.0,1.2,2.5,0.9,35.0,2


In [3]:
X = df.drop('Class', axis=1)
y = df['Class']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train.shape, X_val.shape, y_train.shape, y_val.shape


((211, 11), (53, 11), (211,), (53,))

In [4]:
num = X.select_dtypes(include=['int64', 'float64']).columns
cat = ['Gender']
num, cat

(Index(['Gender', 'AGE', 'Urea', 'Cr', 'HbA1c', 'Chol', 'TG', 'HDL', 'LDL',
        'VLDL', 'BMI'],
       dtype='object'),
 ['Gender'])

In [5]:
num_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('power', PowerTransformer(method='yeo-johnson')),
    ('scaler', StandardScaler())
])
cat_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(transformers=[
    ('num', num_pipe, X_train.select_dtypes(include=['int64', 'float64']).columns),
    ('cat', cat_pipe, X_train.select_dtypes(include=['object', 'category']).columns)
])

In [6]:
X_train = preprocessor.fit_transform(X_train)
X_val = preprocessor.transform(X_val)

# dataset and dataloader

In [7]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val.values, dtype=torch.long)

train_set = TensorDataset(X_train, y_train)
val_set = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32)

feature, target = next(iter(train_loader))
print(feature.shape, target.shape)

torch.Size([32, 77]) torch.Size([32])


# NN

In [8]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes) -> None:
        super().__init__()

        self.fc = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.Mish(),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.Mish(),
            nn.Dropout(0.1),

            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.Mish(),
            nn.Dropout(0.1),

            nn.Linear(32, 16),
            nn.BatchNorm1d(16),
            nn.Mish(),
            nn.Dropout(0.1),

            nn.Linear(16, 8),
            nn.BatchNorm1d(8),
            nn.Mish(),
            nn.Dropout(0.1),

            nn.Linear(8, num_classes)
        )

    def forward(self, X):
        return self.fc(X)

In [9]:
class PLNN(LightningModule):
    def __init__(self, input_size, num_classes=4, learning_rate=1e-3):
        super().__init__()
        self.save_hyperparameters() 
        self.model = NN(input_size=input_size, num_classes=num_classes)
        self.criterion = nn.CrossEntropyLoss()
        self.kappa = MulticlassCohenKappa(num_classes=num_classes, weights='quadratic')
        self.f1 = F1Score(task='multiclass', num_classes=num_classes, average='weighted')
        self.accurasy = Accuracy(task='multiclass', num_classes=num_classes)
    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs) 
        loss = self.criterion(outputs, labels)
        acc = self.accurasy(outputs, labels)
        f1 = self.accurasy(outputs, labels)
        kappa = self.accurasy(outputs, labels)

        
        self.log('train_loss', loss, on_epoch=True, prog_bar=True)
        self.log('train_f1', f1, on_epoch=True, prog_bar=True)
        self.log('train_acc', acc, on_epoch=True, prog_bar=True)
        self.log('train_kappa', kappa, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = self.criterion(outputs, labels)
        acc = self.accurasy(outputs, labels)
        f1 = self.accurasy(outputs, labels)
        kappa = self.accurasy(outputs, labels)


        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
        self.log('val_f1', f1, on_epoch=True, prog_bar=True)
        self.log('val_acc', acc, on_epoch=True, prog_bar=True)
        self.log('val_kappa', kappa, on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=self.hparams.learning_rate)
        return optimizer

# Config

In [10]:
if torch.cuda.is_available():
    accelerator_type = 'gpu'
    devices_to_use = 1
else:
    accelerator_type = 'cpu'
    devices_to_use = 'auto'

checkpoint_callback = ModelCheckpoint(
    monitor='val_f1',
    dirpath='checkpoints/',
    filename='bodyP-{epoch:02d}-{val_f1:.2f}',
    save_top_k=1,
    mode='max'
)
early_stopping = EarlyStopping(
    monitor='val_f1',
    patience=10,
    mode='max'
)
lr_monitor_callback = LearningRateMonitor(logging_interval='epoch')

trainer1 = pl.Trainer(
    max_epochs=300,
    callbacks=[checkpoint_callback, early_stopping, lr_monitor_callback],
    logger=TensorBoardLogger("tb_logs", name="simple_model_experiment"),
    accelerator=accelerator_type,
    devices=devices_to_use,
    log_every_n_steps=10,
    deterministic=True
)

In [11]:
model = PLNN(input_size=X_train.shape[1], num_classes=3, learning_rate=1e-3)
trainer1.fit(model, train_loader, val_loader)

2025-08-28 06:18:45.214156: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756361925.470317      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756361925.542235      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [12]:
trainer1.validate(model, val_loader, ckpt_path='best')


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.9622641801834106     │
│          val_f1           │    0.9622641801834106     │
│         val_kappa         │    0.9622641801834106     │
│         val_loss          │    0.5319299697875977     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 0.5319299697875977,
  'val_f1': 0.9622641801834106,
  'val_acc': 0.9622641801834106,
  'val_kappa': 0.9622641801834106}]